In [ ]:
# k-colouring of a graph
# N = number of nodes
# K = number of colours
'''
In this method we use a colouring matrix:
1 line for each node
1 column for each colour
one and only one 1 in each line, other values 0
It means we need at least NK qubits to describe such a matrix
'''
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
%matplotlib inline
from qiskit.providers.aer import AerError, QasmSimulator
from qiskit.visualization import plot_histogram

backend_state = Aer.get_backend('statevector_simulator')
backend_sim = Aer.get_backend('qasm_simulator')

In [ ]:
## Input section
nNodes=int(input("Number of nodes : "))
nColors=int(input("Number of colours : ")) # nColors <= nNodes

In [ ]:
# Calculation of required Qubits
nc=nColors+1 # For each node, nColors qubits that will be measured,
# + one ancillary
nn2=round((nNodes-1)*nNodes/2) # Number of pairs of different nodes
# and of possible edges
sc=round(nc*nNodes) # Number of qubits to skip before the ancillary qubits
# for pairs and edges
sg=round(nc*nNodes + nn2) # Beginning of the list of qubits that describe the graph
nqbits=sc + 2*nn2 # Total number of qubits

In [ ]:
# Create a Quantum Circuit
q = QuantumRegister(nqbits+1)
c=ClassicalRegister(nColors*nNodes) # To measure to "extract" the colouring matrices
qc = QuantumCircuit(q,c)

In [ ]:
# Importing graph visualization libraries
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

In [ ]:
#Graph adjacency input
# Add the graph (binary list of nNodes*(nNodes-1)/2 elements,
# because the graph is symmetric and there is no i=>i edges.
# Set to |1> the qubits corresponding to an edge.
'''
3 nodes, needs 3 colors
0 1 1
1 0 1
1 1 0
=> 1 1 1
3 nodes, needs 2 colors
0 1 0
1 0 1
0 1 0
=> 1 0 1
'''
count = 0
for i in range(nNodes):
    for j in range(i+1, nNodes):
        if(int(input("vertex {} adjacent to {} ? ( 1 : yes / 0 : no) : ".format(i+1,j+1))) == 1):
            qc.x(sg+count) #set to one if there is edge
            G.add_edge(i+1, j+1)
        count += 1


qc.draw(output='mpl')

colours = []
for node in G:
    colours.append("green")

nx.draw(G,node_color = colours, with_labels = True, font_color = "white")
plt.show()

In [ ]:
# ----------------------------------------Generate a colouring matrix
# Initialisation
# Hadamard gate for qubits that represent the colouring matrix
s=0
for n in range(nNodes):
    for k in range(nColors):
        qc.h(s+k)
    s=s+nc

qc.draw(output='mpl')

In [ ]:
#-----------------------------
# Constraints
# A 1 and only one 1 in each of the coloring matrix
s=0
for n in range(nNodes):
    for k in range(nColors-1):
        for l in range(k+1,nColors):
            # Eliminate 11
            qc.ccx (s+k,s+l,s+nColors)
            qc.cx (s+nColors,s+k)
            qc.reset(s+nColors)
    # Eliminate 0* (no colour assigned to the node n)
    for k in range(nColors):
        qc.x(s+k) # if 0 => 1.
            # Not needed if you can use a negative multicontrolled gate
    cb=list(range(s,s+nColors) )
    qc.mcx (cb,s+nColors)
    for k in range(nColors):
        qc.x(s+k) # if 1 => 0
    qc.cx (s+nColors,s+nColors-1)
    qc.reset(s+nColors)
    s=s+nc
    
qc.barrier()
# At this point, if we measure,
# we find colouring matrices (nNodes lines, nColors columns),
# one and only one 1 in each line (a node does have a colour, and only one)
print('end of colouring matrices')

qc.draw(output='mpl')

In [ ]:
# Switch the ancillary qubits corresponding to pairs of nodes
# that have the same colour
for k in range(nColors):
    s=nc*nNodes
    for n1 in range(nNodes-1):
        for n2 in range(n1+1,nNodes):
            n11=nc*n1+k # If q[n11]=|1> it means the node n1 has the color k
            n22=nc*n2+k # If q[n22]=|1> it means the node n2 has the color k
            qc.ccx(n11,n22,s) # If same color k, set s to |1>.
                # Notice it can happens at most for one k
            s=s+1
#print([n11,n22])
# At this point, if we measure the (nNodes-1)*nNodes)/2 ancillary qubits,
# we get binary strings, in which 1 means "same color" for n1 and n2
print('end of pairs of nodes')
qc.barrier()

qc.draw(output='mpl')

In [ ]:
# Compare to the graph.
for k in range(nColors):
    s=nc*nNodes
    for n1 in range(nNodes-1):
        for n2 in range(n1+1,nNodes):
            n11=nc*n1+k # If q[n11]=|1> it means the node n1 has the color k
            pair = s+nNodes
            ancil = n1*nc + nColors
            cb=[n11,pair,s]
            qc.mcx (cb,ancil)
            cb=[ancil,pair,s]
            qc.mcx (cb,n11)
            qc.reset(ancil)
            qc.barrier()
             # If same color k, set s to |1>.
                # Notice it can happens at most for one k
            s=s+1
print('end of compare to graph')

qc.draw(output='mpl')

In [ ]:
#Filter 0*
# A 1 and only one 1 in each of the coloring matrix
s=0
for n in range(nNodes):
    # Eliminate 0* (no colour assigned to the node n)
    
    cb=list(range(s,s+nColors))
 
    qc.x(cb)
    qc.mcx (cb,s+nColors)
    qc.x(cb)
    pos=0
    for m in range(nNodes):
        for k in range(nColors):
            qc.reset(nqbits)
            qc.cswap(s+nColors,pos+k,nqbits)
        qc.barrier()
        pos=pos+nc
    s=s+nc

qc.barrier()

qc.draw(output='mpl')

In [ ]:

# Measure (only the qubits describing the colouring matrices)
cb=0
for n in range(nNodes):
    s=n*(nColors+1)
    for k in range(nColors):
        qb=s+k
        qc.measure(qb,cb)
        cb=cb+1
print('end of measures')

qc.draw(output='mpl')

In [ ]:
# Quick on small graphs, but memory error for 4 nodes, 3 colours
job = execute(qc, backend_sim,shots=1000)
result=job.result()
#print(result)
print('end of execute')

In [ ]:
# Grab the results from the job.
counts = result.get_counts(qc)

if len(counts) > 1:
    counts.pop("0"*nNodes*nColors, None)
    
plot_histogram(counts, color='midnightblue', title="Colourings")

In [ ]:
#displaying graph colourings
colour = ["blue", "red", "green", "purple"]

for graph in counts:
    print("Graph for colouring {}".format(graph))
    colours = []
    cList =list(map(''.join, zip(*[iter(graph)]*nColors)))
    for node in cList:
        colours.append(colour[node.index("1")])
    nx.draw(G,node_color = colours, with_labels = True, font_color = "white")
    plt.show()